In [4]:
import pandas as pd
import ast

# Cargar el dataset
file_path = "primer_trabajo/movies_dataset.csv"  

# Cargar el archivo CSV
try:
    df = pd.read_csv("movies_dataset.csv")
except FileNotFoundError:
    raise FileNotFoundError(f"El archivo {file_path} no se encontró.")


FileNotFoundError: El archivo primer_trabajo/movies_dataset.csv no se encontró.

In [2]:
# 1. Desanidar los campos anidados
anidado_cols = ["belongs_to_collection", "production_companies"]
for col in anidado_cols:
    if col in df.columns:
        # Convertir cadenas en listas/diccionarios
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})
        
        # Extraer nombres de las claves principales (por ejemplo, 'name')
        if col == "belongs_to_collection":
            df[col + "_name"] = df[col].apply(lambda x: x.get("name") if isinstance(x, dict) else None)
        elif col == "production_companies":
            df[col + "_names"] = df[col].apply(lambda x: ", ".join([i.get("name", "") for i in x]) if isinstance(x, list) else None)

In [6]:
# 2. Reemplazar valores nulos en revenue y budget por 0 y convertirlos a numeros
df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce").fillna(0)
df["budget"] = pd.to_numeric(df["budget"], errors="coerce").fillna(0)

# 3. Eliminar filas con valores nulos en release_date
df = df.dropna(subset=["release_date"])


In [9]:
# 4. Formatear fechas y extraer el año
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df = df.dropna(subset=["release_date"])  # Eliminar fechas no válidas
df["release_date"] = df["release_date"].dt.strftime("%Y-%m-%d")
df["release_year"] = pd.to_datetime(df["release_date"]).dt.year



In [7]:
# 5. Crear la columna return (retorno de inversión)
def calcular_retorno(revenue, budget):
    if budget > 0:
        return revenue / budget
    return 0
df["return"] = df.apply(lambda row: calcular_retorno(row["revenue"], row["budget"]), axis=1)

In [8]:
# 6. Eliminar columnas no necesarias
columns_to_drop = ["video", "imdb_id", "adult", "original_title", "poster_path", "homepage"]
df = df.drop(columns=columns_to_drop, errors="ignore")

# Guardar el dataset limpio
output_file = "dataset_cleaned.csv"
df.to_csv(output_file, index=False)
print(f"El dataset limpio se ha guardado como {output_file}.")


El dataset limpio se ha guardado como dataset_cleaned.csv.
